In [1]:
import os

In [2]:
os.chdir("/Users/vanshbansal/Desktop/Road Accidents")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir : Path
    results : Path
    preprocessed_x_train : Path
    preprocessed_x_test : Path
    preprocessed_y_train : Path
    preprocessed_y_test : Path

In [4]:
from src.constants import CONFIG_FILE_PATH
from src.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self , config_file_path = CONFIG_FILE_PATH):
        config = read_yaml(config_file_path)

        self.config = config

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            results = config.results,
            preprocessed_x_train = config.preprocessed_x_train,
            preprocessed_x_test = config.preprocessed_x_test,
            preprocessed_y_train = config.preprocessed_y_train,
            preprocessed_y_test = config.preprocessed_y_test,
        )

        return model_trainer_config


In [9]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
#from ydata_profiling import ProfileReport
from sklearn.metrics import accuracy_score
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier

class ModelTrainer:
    def __init__(self , config: ModelTrainerConfig):
        self.config = config
    
    def log_metrics(self , y_test , y_pred , model_name:str):
        # Calculate metrics
        acc_score = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')  
        recall = recall_score(y_test, y_pred, average='weighted')        
        f1 = f1_score(y_test, y_pred, average='weighted') 

        metrics_file_path = self.config.results  
        with open(metrics_file_path, 'a') as file:
            file.write(f"Model Evaluation Metrics for {model_name} model:\n")
            file.write(f"Accuracy: {acc_score:.4f}\n")
            file.write(f"Precision: {precision:.4f}\n")
            file.write(f"Recall: {recall:.4f}\n")
            file.write(f"F1 Score: {f1:.4f}\n")
            file.write(f"\n\n")


    def LogisticRegression(self , x_train,y_train , x_test , y_test):
        lr = LogisticRegression()
        lr.fit(x_train , y_train)

        y_pred = lr.predict(x_test)

        self.log_metrics(y_test , y_pred , "Logistic Regression")
    
    def KNN(self , x_train , y_train , x_test , y_test):
        knn = KNeighborsClassifier()
        knn.fit(x_train , y_train)

        y_pred = knn.predict(x_test)

        self.log_metrics(y_test , y_pred , "KNN")

    def start_model_training(self):
        x_train = np.load(self.config.preprocessed_x_train)
        x_test = np.load(self.config.preprocessed_x_test)
        y_train = np.load(self.config.preprocessed_y_train)
        y_test = np.load(self.config.preprocessed_y_test)

        self.LogisticRegression(x_train , y_train , x_test , y_test)
        self.KNN(x_train , y_train , x_test , y_test)


In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.start_model_training()
except Exception as e:
    raise e

[2025-01-11 13:51:06,926: INFO: common: yaml file: config.yaml loaded successfully]
[2025-01-11 13:51:06,928: INFO: common: created directory at: artifacts/model_trainer]


/Users/vanshbansal/Desktop/Road Accidents/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/vanshbansal/Desktop/Road Accidents/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
